<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/brace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Github/capstone/brace

/content/drive/MyDrive/Github/capstone/brace


In [3]:
%ls

 annotations/      dataset/            'MODELS ACCURACY.gdoc'   train_subset.pkl
 bracedataset.py   dataset_pytorch.py   path/                   utils/
 brace_test.pkl    figures/             __pycache__/            videos_info.csv
 brace_test.pth    keras_model/         README.md
 brace_train.pkl   lstm_model2.pth      tensor_dataset.pkl
 brace_train.pth   lstm_model.pth       test_subset.pkl


In [4]:
# !nvidia-smi

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone/brace')

In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import bracedataset
from pathlib import Path
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import TensorDataset
from tqdm import tqdm
from pathlib import PosixPath
import pickle
import re
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.optim.lr_scheduler import ExponentialLR
import json
from pathlib import Path

In [52]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping

Obtain BRACE dataset clips and labels, only need to to this once, they are saved for future reference


In [7]:
# #load paths
# sequences_path = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')
# pose_jsons = list(Path(sequences_path).rglob('**/*.json'))
# segments= pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/segments.csv'))
# #get clips
# dataClips, labels=bracedataset.BraceDataset(segments, pose_jsons)
# #encode labels from 0-2
# unique_categories = list(set(labels))
# category_to_int = {category: index for index, category in enumerate(unique_categories)}
# encoded_categories = [category_to_int[category] for category in labels]

# #get max length of the keypoints
# max_length = max(seq.shape[0] for seq in dataClips)

# #pad tensor so they are of the same lenght
# dataClips_tensors=[np.pad(array.astype(np.float32), ((0, max_length - array.shape[0]), (0, 0), (0, 0)), 'constant', constant_values=0) for array in dataClips]
# dataClips_tensors=torch.tensor(dataClips_tensors)
# #convert labels to tensors
# label_tensors = np.array(encoded_categories)
# label_tensors=torch.tensor(label_tensors)
# ##SPLIT SET FOR REPRODUCTIVITY
# dataset = TensorDataset(dataClips_tensors, label_tensors)
# # Set seed for reproducibility
# torch.manual_seed(42)

# # Determine split sizes
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size

# # Split dataset
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# print(f"Training set size: {len(train_dataset)}")
# print(f"Testing set size: {len(test_dataset)}")
# # Save the TensorDataset
# with open('tensor_dataset.pkl', 'wb') as f:
#     pickle.dump(dataset, f)

# # Save the Subset
# with open('train_subset.pkl', 'wb') as f:
#     pickle.dump(train_dataset, f)
# with open('test_subset.pkl', 'wb') as f:
#     pickle.dump(test_dataset, f)

Load previously saved dataset

In [8]:
# Load the TensorDataset
with open('tensor_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

# Load the training Subset
with open('train_subset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

# Load the testing Subset
with open('test_subset.pkl', 'rb') as f:
    test_dataset = pickle.load(f)

In [9]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Use this to load the dataset

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes=3):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
# Initialize the model
input_size = 17 * 2
hidden_size = 128
num_layers = 2
num_classes = 3

model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 10
for epoch in range(num_epochs):

    for sequences, labels in train_loader:
        sequences = sequences.to(device).view(-1, 1280, 34)  # Adjust shape for LSTM
        labels = labels.to(device)

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# model_path = "lstm_model2.pth"
# torch.save(model.state_dict(), model_path)
# print(f"Model saved to {model_path}")

In [ ]:
# model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)
# model.load_state_dict(torch.load(model_path))


# print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device).view(-1, 1280, 34)
        labels = labels.to(device)
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')
#accuracy 33.85%

CNN model with RNN layer


In [15]:
#load paths
sequences_path = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')
pose_jsons = list(Path(sequences_path).rglob('**/*.json'))
segments= pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/segments.csv'))
#get clips
dataClips, labels, names=bracedataset.BraceDataset(segments, pose_jsons)
#encode labels from 0-2
unique_categories = list(set(labels))
category_to_int = {category: index for index, category in enumerate(unique_categories)}
encoded_categories = [category_to_int[category] for category in labels]

In [14]:
segments[segments['uid']=='3rIk56dcBTM.3315.3608']

,video_id,seq_idx,start_frame,end_frame,dance_type,dance_type_id,dancer,dancer_id,year,uid
9,3rIk56dcBTM,2,3315,3608,footwork,2,el niño,10,2011,3rIk56dcBTM.3315.3608


In [28]:
flattened_clips = [clip.reshape(clip.shape[0], -1) for clip in dataClips]

padded_clips = pad_sequences(flattened_clips, padding='post', dtype='float32')


In [29]:
### KERAS CNN MODEL######

X_train, X_test, y_train, y_test = train_test_split(
    padded_clips, encoded_categories, test_size=0.20, random_state=42)
# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

# Define the model
model = Sequential([
    LSTM(50, input_shape=(padded_clips.shape[1], padded_clips.shape[2])),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_train, y_train_onehot, epochs=10, batch_size=32, validation_split=0.2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                17000     
                                                                 
 dense (Dense)               (None, 100)               5100      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 22403 (87.51 KB)
Trainable params: 22403 (87.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
27/27 [==============================] - 18s 536ms/step - loss: 1.0997 - accuracy: 0.3113 - val_loss: 1.0991 - val_accuracy: 0.2949
Epoch 2/1

In [30]:
# model.save('/content/drive/MyDrive/Github/capstone/brace/keras_model')
# Load the model
loaded_model = load_model('path/to/location/keras_model')

# test
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test_onehot)
print(f'Test Accuracy: {test_accuracy}')

9/9 [==============================] - 2s 131ms/step - loss: 1.0955 - accuracy: 0.3616
Test Accuracy: 0.361623615026474


INCLUDE FEATURES IN THE MODEL

In [44]:
#sort additional features
features= segments.copy()
features['uid']= pd.Categorical(features['uid'], categories=names, ordered=True)
features= features.sort_values(by='uid')
features['frames']= features['end_frame']- features['start_frame']


,video_id,seq_idx,start_frame,end_frame,dance_type,dance_type_id,dancer,dancer_id,year,uid,frames
9,3rIk56dcBTM,2,3315,3608,footwork,2,el niño,10,2011,3rIk56dcBTM.3315.3608,293
7,3rIk56dcBTM,2,3012,3166,toprock,0,el niño,10,2011,3rIk56dcBTM.3012.3166,154


In [27]:
# One-hot encode
preprocessor = ColumnTransformer(
    transformers=[
        ('seq', StandardScaler(), ['seq_idx']),
        ('cat', OneHotEncoder(), ['dancer_id']),
        ('num', StandardScaler(), ['year']),

    ]
)

features_preprocessed = preprocessor.fit_transform(features[['seq_idx', 'dancer_id', 'year']])
features_preprocessed = features_preprocessed.toarray()
features_preprocessed.shape

(1352, 66)

In [35]:
# split train, test
X_train_clips, X_test_clips, X_train_features, X_test_features, y_train, y_test = train_test_split(
    padded_clips, features_preprocessed, encoded_categories, test_size=0.2, random_state=42)


y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

In [36]:
# Keypoint input branch
keypoint_input = Input(shape=(X_train_clips.shape[1], X_train_clips.shape[2]), name='keypoint_input')
keypoint_model = LSTM(50)(keypoint_input)
keypoint_model = Dense(100, activation='relu')(keypoint_model)

# Additional features input branch
additional_features_input = Input(shape=(X_train_features.shape[1],), name='features_input')
features_model = Dense(50, activation='relu')(additional_features_input)

# Concatenate the outputs of the two branches
combined = concatenate([keypoint_model, features_model])

# Add a classification layer
combined = Dense(100, activation='relu')(combined)
combined = Dropout(0.5)(combined)
output = Dense(3, activation='softmax')(combined)  # Assuming 3 classes

# Build the model
model = Model(inputs=[keypoint_input, additional_features_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to check architecture
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 keypoint_input (InputLayer  [(None, 1280, 34)]           0         []                            
 )                                                                                                
                                                                                                  
 lstm_2 (LSTM)               (None, 50)                   17000     ['keypoint_input[0][0]']      
                                                                                                  
 features_input (InputLayer  [(None, 66)]                 0         []                            
 )                                                                                                
                                                                                            

In [37]:
history = model.fit([X_train_clips, X_train_features], y_train_onehot, validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10
27/27 [==============================] - 23s 695ms/step - loss: 1.1060 - accuracy: 0.3183 - val_loss: 1.1051 - val_accuracy: 0.2627
Epoch 2/10
27/27 [==============================] - 13s 501ms/step - loss: 1.0952 - accuracy: 0.3692 - val_loss: 1.1050 - val_accuracy: 0.2719
Epoch 3/10
27/27 [==============================] - 13s 476ms/step - loss: 1.0885 - accuracy: 0.4097 - val_loss: 1.1059 - val_accuracy: 0.2949
Epoch 4/10
27/27 [==============================] - 14s 523ms/step - loss: 1.0876 - accuracy: 0.3738 - val_loss: 1.1105 - val_accuracy: 0.2949
Epoch 5/10
27/27 [==============================] - 16s 594ms/step - loss: 1.0816 - accuracy: 0.4005 - val_loss: 1.1077 - val_accuracy: 0.3134
Epoch 6/10
27/27 [==============================] - 19s 701ms/step - loss: 1.0777 - accuracy: 0.4051 - val_loss: 1.1067 - val_accuracy: 0.3410
Epoch 7/10
27/27 [==============================] - 13s 487ms/step - loss: 1.0668 - accuracy: 0.4144 - val_loss: 1.1151 - val_accuracy: 0.3226

In [42]:
model.save('/content/drive/MyDrive/Github/capstone/brace/k_model_features')
# Load the model
loaded_model = load_model('/content/drive/MyDrive/Github/capstone/brace/k_model_features')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate([X_test_clips, X_test_features], y_test_onehot)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 147ms/step - loss: 1.1320 - accuracy: 0.3173
Test Loss: 1.1320452690124512
Test Accuracy: 0.31734317541122437


CHANGE FEATURES TO SEQUENCE AND CLIP LENGHT, TO TEST IF THERE IS LESS OVERFITTING

In [45]:
# One-hot encode
preprocessor = ColumnTransformer(
    transformers=[
        ('seq', StandardScaler(), ['seq_idx']),
        ('frames', OneHotEncoder(), ['frames'])
    ]
)

features_preprocessed = preprocessor.fit_transform(features[['seq_idx', 'frames']])
features_preprocessed = features_preprocessed.toarray()
features_preprocessed.shape

(1352, 498)

In [ ]:
# #### KERAS FEATURE MODEL2#####

# X_train_clips, X_test_clips, X_train_features, X_test_features, y_train, y_test = train_test_split(
#     padded_clips, features_preprocessed, encoded_categories, test_size=0.2, random_state=42)


# y_train_onehot = to_categorical(y_train, num_classes=3)
# y_test_onehot = to_categorical(y_test, num_classes=3)

# # Keypoint input branch
# keypoint_input = Input(shape=(X_train_clips.shape[1], X_train_clips.shape[2]), name='keypoint_input')
# keypoint_model = LSTM(50)(keypoint_input)
# keypoint_model = Dense(100, activation='relu')(keypoint_model)

# # Additional features input branch
# additional_features_input = Input(shape=(X_train_features.shape[1],), name='features_input')
# features_model = Dense(50, activation='relu')(additional_features_input)

# # Concatenate the outputs of the two branches
# combined = concatenate([keypoint_model, features_model])

# # Add a classification layer
# combined = Dense(100, activation='relu')(combined)
# combined = Dropout(0.5)(combined)
# output = Dense(3, activation='softmax')(combined)  # Assuming 3 classes

# # Build the model
# model = Model(inputs=[keypoint_input, additional_features_input], outputs=output)

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Model summary to check architecture
# model.summary()

In [51]:

X_train_clips, X_test_clips, X_train_features, X_test_features, y_train, y_test = train_test_split(
    padded_clips, features_preprocessed, encoded_categories, test_size=0.2, random_state=42)


y_train_onehot = to_categorical(y_train, num_classes=3)
y_test_onehot = to_categorical(y_test, num_classes=3)

# Keypoint input branch
keypoint_input = Input(shape=(X_train_clips.shape[1], X_train_clips.shape[2]), name='keypoint_input')
keypoint_model = LSTM(50, recurrent_regularizer=l1_l2(l1=0.01, l2=0.01))(keypoint_input)  # Adding regularization to LSTM
keypoint_model = Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(keypoint_model)  # Adding regularization
keypoint_model = Dropout(0.5)(keypoint_model)  # Adding dropout

# Additional features input branch
additional_features_input = Input(shape=(X_train_features.shape[1],), name='features_input')
features_model = Dense(50, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(additional_features_input)  # Adding regularization
features_model = Dropout(0.5)(features_model)  # Adding dropout

# Concatenate the outputs of the two branches
combined = concatenate([keypoint_model, features_model])

# Add a classification layer
combined = Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(combined)  # Adding regularization
combined = Dropout(0.5)(combined)  # Adding dropout
output = Dense(3, activation='softmax')(combined)  # Assuming 3 classes

# Build the model
model = Model(inputs=[keypoint_input, additional_features_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to check architecture
model.summary()



Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 keypoint_input (InputLayer  [(None, 1280, 34)]           0         []                            
 )                                                                                                
                                                                                                  
 lstm_4 (LSTM)               (None, 50)                   17000     ['keypoint_input[0][0]']      
                                                                                                  
 features_input (InputLayer  [(None, 498)]                0         []                            
 )                                                                                                
                                                                                            

In [53]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    [X_train_clips, X_train_features], y_train_onehot,
    validation_data=([X_test_clips, X_test_features], y_test_onehot),
    epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
34/34 [==============================] - 28s 714ms/step - loss: 30.7461 - accuracy: 0.3358 - val_loss: 22.2211 - val_accuracy: 0.3542
Epoch 2/100
34/34 [==============================] - 34s 1s/step - loss: 16.2763 - accuracy: 0.3525 - val_loss: 10.8972 - val_accuracy: 0.3616
Epoch 3/100
34/34 [==============================] - 20s 597ms/step - loss: 7.5851 - accuracy: 0.3228 - val_loss: 4.8566 - val_accuracy: 0.3616
Epoch 4/100
34/34 [==============================] - 24s 686ms/step - loss: 3.4924 - accuracy: 0.3321 - val_loss: 2.3563 - val_accuracy: 0.3616
Epoch 5/100
34/34 [==============================] - 29s 856ms/step - loss: 1.8171 - accuracy: 0.3191 - val_loss: 1.4518 - val_accuracy: 0.3616
Epoch 6/100
34/34 [==============================] - 29s 866ms/step - loss: 1.3050 - accuracy: 0.3599 - val_loss: 1.2047 - val_accuracy: 0.3616
Epoch 7/100
34/34 [==============================] - 25s 733ms/step - loss: 1.1854 - accuracy: 0.3377 - val_loss: 1.1714 - val_accuracy

In [48]:
model.save('/content/drive/MyDrive/Github/capstone/brace/k_model_features3')
# Load the model
loaded_model = load_model('/content/drive/MyDrive/Github/capstone/brace/k_model_features3')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate([X_test_clips, X_test_features], y_test_onehot)
print(f"Model name: {loaded_model}")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 137ms/step - loss: 1.3042 - accuracy: 0.3321
Test Loss: 1.304213285446167
Test Accuracy: 0.33210331201553345
